In [1]:
import pandas as pd
import numpy as np
import requests
df = pd.read_csv('成都写字楼数据.csv',error_bad_lines=False)

In [2]:
df.head()

,building_type,building_year,chuzumianji,floor,loupan_content,loupandizhi,price_perunit_perday,release_date,service_price,titile,total_price_10K_permonth,url,xiezilou_level,xzl_content,zhuangxiu
0,纯写字楼,NaN,75㎡,高区(共42层),\r\n \r\n \r\n ...,"['大源', '蜀都中心']",82.67,2019/9/6 13:46:45,NaN,小面积75平丨1老板室8卡丨房东直租仅需6200无加班费,6200,https://cd.office.fang.com/zu/3_208853366.html,甲级,\r\n\t\t\t\t\t\r\n\t\t\t\t\t\t房源亮点\r\n\t\t\t\t...,精装修(可分割)
1,纯写字楼,NaN,300㎡,低区(共60层),\r\n \r\n \r\n ...,"['金融城', '中国华商金融中心']",80,2019/8/22 17:17:46,物业费23.00元/平米·月,精致型清新风格满足您享受每个工作日 方正落地窗 4隔间,24000,https://cd.office.fang.com/zu/3_205730066.html,甲级,\r\n\t\t\t\t\t\r\n\t\t\t\t\t\t房源亮点\r\n\t\t\t\t...,精装修(可分割)
2,纯写字楼,NaN,352㎡,高区(共46层),\r\n \r\n \r\n ...,"['高新区', '东方希望天祥广场']",80,2019/7/19 15:17:19,物业费17.00元/平米·月,云端视野！天祥广场 使用面积700平 地铁直达 商圈成熟 随时看房,28160,https://cd.office.fang.com/zu/3_208284760.html,甲级,\r\n\t\t\t\t\t\r\n\t\t\t\t\t\t房源亮点\r\n\t\t\t\t...,精装修(可分割)
3,纯写字楼,NaN,86㎡,中区(共18层),\r\n \r\n \r\n ...,"['神仙树', '英祥财富领域']",44.19,2019/7/17 10:09:43,物业费3.80元/平米·月,高新区火车南站地铁口 富森美 凯德广场 苏宁 精装跃层写字楼急租,3800,https://cd.office.fang.com/zu/3_208256024.html,甲级,\r\n\t\t\t\t\t\r\n\t\t\t\t\t\t房源亮点\r\n\t\t\t\t...,精装修(可分割)
4,所在楼层,NaN,260㎡,中区(共33层),\r\n \r\n \r\n ...,"['天府广场', '雄飞中心']",70,2019/9/10 17:56:35,NaN,地铁口恒大雄飞中心带家具30卡位双面采光电梯口,18200,https://cd.office.fang.com/zu/3_208909153.html,甲级,\r\n\t\t\t\t\t\r\n\t\t\t\t\t\t房源亮点\r\n\t\t\t\t...,精装修(不可分割)


In [3]:
def replacestr(string):
    string = string.replace('[','')
    string = string.replace(']','')
    string = string.replace("'",'')
    return string

df['loupandizhi'] = df['loupandizhi'].apply(replacestr)

list_location = df['loupandizhi'].value_counts().index.tolist()

dict_location = {}
Key = '6b95b84cab6100c0a6f52c62ece97e4b'
My_key = 'f654d644aeae0e1da847d7ca408f4370'
url_pre = 'https://restapi.amap.com/v3/geocode/geo?key={}&address={}&city={}'
counter = 0
for i in list_location:
    location = i.split(",")[-1].strip()
    url = url_pre.format(Key, location, '成都')
    res = requests.get(url).json()
    if res['status'] == '0':
        dict_location[i] = np.nan
        counter +=1
    else:
        try: 
            dict_location[i] = res['geocodes'][0]['location']
            if counter % 250 == 0:
                print('one batch(250) processed')
        except:
            dict_location[i] = np.nan
            
        counter +=1

one batch(250) processed
one batch(250) processed
one batch(250) processed


In [6]:
df['location_coords'] = np.nan

counter = 0
for i in list_location:
    counter += 1
    df_temp = df[df['loupandizhi'] == i]
    df_temp_index = df_temp.index.tolist()
    df.iloc[df_temp_index,15] = dict_location[i]
    if counter %50 == 0:
        print('A batch(50) is done')

A batch(50) is done
A batch(50) is done
A batch(50) is done
A batch(50) is done
A batch(50) is done
A batch(50) is done
A batch(50) is done
A batch(50) is done
A batch(50) is done
A batch(50) is done


In [8]:
df.to_csv('成都写字楼出租地址数据.csv', encoding = 'utf-8-sig',index = False)